In [15]:
import json
import os
import PyPDF2
from openai import OpenAI

# Paths to files
resume_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\ganeshrajk_resume_Sept.pdf"
json_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\questions.json"
detailed_resume_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\resume_detailed.txt"
secrets_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\user_files\secrets.json"

# Load API key from secrets file
with open(secrets_path, 'r') as file:
    data = json.load(file)
OPENAI_API_KEY = data["OPENAI_KEY"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Initialize OpenAI client
client = OpenAI()

# Function to extract text from PDF file (resume)
def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Function to load JSON file (Q&A data)
def load_json(json_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

# Function to generate detailed summary using GPT-4
def generate_detailed_summary(pdf_text, json_data, openai_api_key):
    # Combine PDF text and JSON data into a detailed prompt
    prompt = f"""
    You are given a resume of a job applicant. 
    Extract as much information from the resume as possible. 
    Analyze the resume thoroughly and rewrite the resume in a more detailed and elaborated format. 
    Also generate all possible questions and answers based on the information provided in the resume. 
    PDF (Resume) Content:
    {pdf_text}

    JSON (Q&A) Data:
    {json.dumps(json_data, indent=2)}
    """
    
    # Call GPT-4 using the new OpenAI API syntax
    response = client.chat.completions.create(
        model="gpt-4",  # Change the model to "gpt-4" if needed
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes job applicant profiles."},
            {"role": "user", "content": prompt},
        ]
    )
    
    # Extract and return the generated summary
    return response.choices[0].message.content

# Main function to process PDF, JSON, and generate the detailed summary
def main(pdf_file_path, json_file_path, openai_api_key, output_file_path=detailed_resume_path):
    # Extract text from the PDF (resume)
    pdf_text = extract_text_from_pdf(pdf_file_path)
    
    # Load the Q&A data from the JSON file
    json_data = load_json(json_file_path)
    
    # Generate a detailed profile summary using GPT-4
    detailed_summary = generate_detailed_summary(pdf_text, json_data, openai_api_key)
    
    # Save the generated summary to a text file
    with open(output_file_path, 'w') as file:
        file.write(detailed_summary)
    
    print(f"Detailed summary saved to {output_file_path}")

# Example usage
if __name__ == "__main__":
    pdf_file = resume_path  # Path to the resume PDF file
    json_file = json_path   # Path to the Q&A JSON file
    openai_api_key = OPENAI_API_KEY  # Replace with your OpenAI API key
    
    # Generate and save the summary
    main(pdf_file, json_file, openai_api_key)


Detailed summary saved to C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\resume_detailed.txt
